In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
import pingouin as pg
sns.set_context('paper')
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

In [2]:
def simu_success(tag, params):
    # which task
    path = "simuS1_data/simuS1_design.pkl"
    if tag == "Asso-CR":
        # path = "../Data/simuS1_group1_design.pkl"
        nitems = 48*4  # 96
        test1_num = 40
        i = 1
    elif tag == "Pair-CR":
        # path = "../Data/simuS1_group2_design.pkl"
        nitems = 48*4  # 176
        test1_num = 80
        i = 2
    elif tag == "Item-CR":
        # path = "../Data/simuS1_group3_design.pkl"
        nitems = 48*4  # 136
        test1_num = 80
        i = 3

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    df_study = df_study.query(f"group == {i}")
    df_test = df_test.query(f"group == {i}")

    # load semantic matrix
    s_mat = np.load('../../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','list','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort
        nlists = len(np.unique(df_simu.list))

        for lst in range(nlists):
            tmp_corr = df_simu.query(f"session == {sess} and list == {lst}")["correct_ans"][test1_num:]
            corrid = np.searchsorted(tmp, tmp_corr)
            corr_fin = [f_in_dif[sess][lst*int(test1_num/2)+i][id] for i, id in enumerate(corrid)]
            omax_fin = [np.max(np.delete(f_in_dif[sess][lst*int(test1_num/2)+i], id)) for i, id in enumerate(corrid)]

            tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
            tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    # optimal threshold for test1 (only when manually!!)
    # csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    # opt_thresh = np.mean(csim_two)
    # df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

In [3]:
# anal_perform used by pso
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_S1 as anal_perform

In [4]:
# def anal_perform(df_simu):

#     # get correctness
#     df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

#     # recognition performance
#     df_recog = df_simu.query("test==1")
#     hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
#     hr = hr_far['Yes rate'][1]
#     far = hr_far['Yes rate'][0]
#     z_hr_far = sp.stats.norm.ppf(hr_far)
#     d_prime = z_hr_far[1].item() - z_hr_far[0].item()
#     print("recognition: \n", hr_far)
#     print("d_prime: ", d_prime)

#     # cued recall performance
#     df_cr = df_simu.query("test==2")
#     p_rc = df_cr.correct.mean()
#     print("cued recall: \n", p_rc)

#     # analyze pair
#     def get_pair(df_tmp):
#         df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
#         df_tmp_pair.columns = ["test1","test2"]
#         df_tmp_pair.reset_index(inplace=True)
#         return df_tmp_pair

#     df_simu_p = df_simu.query("pair_idx >= 0")
#     df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
#     test2_rsp = pd.Categorical(df_pair.test2, categories=[0,1])
#     test1_rsp = pd.Categorical(df_pair.test1, categories=[0,1])
#     df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)
#     print("contingency table: \n", df_tab)
#     # print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

#     # compute" Q
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
#     q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
#     print("Q: ", q)

#     return p_rc, hr, far, q

### Study Params

In [5]:
params = cmr.make_default_params()
# learn while retrieving
# multiple lists, so beta_rec_post
# correct session num - done
# it seems that the size of all potential words matters, the more, the worse CR. RISK IT!

In [6]:
# # manually adjusted params
# # study params should be the same across three groups
# params.update(
#     beta_enc = 0.3,
#     gamma_fc = 0.2,
#     gamma_cf = 0.2,
#     s_fc = 0.2,
#     s_cf = 0.2,
#     phi_s = 1,
#     phi_d = 0.6,
#     d_ass = 1,
#     learn_while_retrieving = False,
#     var_enc = 1,
#     bad_enc_ratio = 1,
# )
# params.update(
#     beta_cue = 0.4,
#     beta_rec = 0.1,  # beta for retrieved item
#     beta_rec_post = 1,
#     beta_distract = 0.1,
#     # beta_rec_post = 0.1,
#     c_thresh_itm = 0.01,
#     c_thresh_ass = 0.01,
#     c_thresh = 0.01,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params

In [7]:
# pso results
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='S1')
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/S1_231102_500-200.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
# params.update(beta_rec_post=1)
params.update(learn_while_retrieving=True, rec_time_limit=10000)
params

{'beta_enc': 0.441868,
 'beta_rec': 0.03125,
 'beta_cue': 0.74583,
 'beta_rec_post': 0.92021,
 'beta_distract': 0.467233,
 'phi_s': 4.801034,
 'phi_d': 2.025982,
 's_cf': 0.016863,
 's_fc': 0.218485,
 'kappa': 0.366019,
 'eta': 0.020671,
 'omega': 9.192145,
 'alpha': 0.713581,
 'c_thresh': 0.297338,
 'c_thresh_itm': 0.410577,
 'c_thresh_ass': 0.530453,
 'd_ass': 0.63822,
 'lamb': 0.069884,
 'rec_time_limit': 10000,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.209973,
 'gamma_cf': 0.709904}

### Association - CR (Group1)

In [8]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params)
df_simu_g1

CMR2 Time: 130.9423487186432


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,1213,926,1.0,75.420311,0.591292,1,ROUGE,MOUTH,0,1,-1,0,1,-1.000000,-1.000000
1,0,0,830,717,0.0,304.645386,0.521488,1,LIST,HORIZON,1,1,34,0,1,-1.000000,-1.000000
2,0,0,577,1183,0.0,1776.684326,0.433321,1,FLEA,RELISH,1,1,0,0,1,-1.000000,-1.000000
3,0,0,588,439,1.0,87.677292,0.583763,1,FOLDER,DANDRUFF,1,0,25,0,1,-1.000000,-1.000000
4,0,0,687,1311,1.0,40.472012,0.622415,1,HANGER,SIRLOIN,1,1,22,0,1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99,4,1169,-1,595.0,750.000000,0.781864,2,RAT,-1,331,0,226,99,1,-0.196618,6.179448
29996,99,4,1349,-1,595.0,720.000000,0.794742,2,SPHINX,-1,848,1,219,99,1,0.658614,6.781781
29997,99,4,1307,-1,595.0,740.000000,0.769630,2,SILK,-1,72,0,194,99,1,0.615654,6.814439
29998,99,4,984,-1,595.0,720.000000,0.741289,2,ORGAN,-1,899,1,196,99,1,0.711375,6.599813


In [9]:
subjects = np.unique(df_simu_g1.subject)
g1_stats = []
for subj in subjects:
    df_subj = df_simu_g1.query(f"subject == {subj}").copy()
    # df_subj = df_simu_g1.query(f"subject == {subj} and list == 0").copy()
    g1_stats.append(list(anal_perform(df_subj)))

In [10]:
print(np.array(g1_stats))
print(np.mean(np.array(g1_stats),axis=0))
print(sp.stats.sem(np.array(g1_stats),axis=0))

[[0.57       0.84       0.14       0.83210017]
 [0.31       0.76       0.19       0.82263191]
 [0.32       0.8        0.23       0.78431373]
 [0.47       0.77       0.19       0.84662577]
 [0.34       0.76       0.12       0.84692082]
 [0.57       0.81       0.17       0.92939548]
 [0.3        0.8        0.13       0.76321839]
 [0.32       0.81       0.17       0.92482916]
 [0.31       0.81       0.12       0.59459459]
 [0.36       0.76       0.18       0.8611266 ]
 [0.44       0.86       0.12       0.8       ]
 [0.29       0.82       0.19       0.90655022]
 [0.33       0.72       0.07       0.7789025 ]
 [0.51       0.86       0.17       0.73684211]
 [0.57       0.85       0.14       0.89751013]
 [0.33       0.88       0.18       0.63031625]
 [0.44       0.79       0.1        0.88431062]
 [0.44       0.81       0.15       0.76557864]
 [0.62       0.89       0.19       0.77255204]
 [0.36       0.82       0.12       0.64880521]
 [0.35       0.82       0.24       0.78492239]
 [0.33       

In [11]:
g1_stats = np.array(g1_stats)
g1_ds = sp.stats.norm.ppf(np.array(g1_stats[:,1])) - sp.stats.norm.ppf(np.array(g1_stats[:,2]))
print(np.mean(g1_ds[np.isfinite(g1_ds)]))
print(sp.stats.sem(g1_ds[np.isfinite(g1_ds)]))

1.9170377952339734
0.02196960828880541


### Pair - CR (Group2)

In [42]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)
df_simu_g2

CMR2 Time: 119.15219926834106


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,192,1402,1.0,213.237717,0.539325,1,BULB,SUIT,1,1,13,0,2,-1.000000,-1.000000
1,0,0,810,1481,0.0,1147.070679,0.455197,1,LEMON,TOOTH,0,-1,-1,0,2,-1.000000,-1.000000
2,0,0,384,686,1.0,10.579496,0.689500,1,COTTAGE,HANDKERCHIEF,1,1,8,0,2,-1.000000,-1.000000
3,0,0,1317,1137,0.0,19.992231,0.163475,1,SKILLET,PUBLISHER,0,-1,-1,0,2,-1.000000,-1.000000
4,0,0,506,977,0.0,2702.760742,0.408810,1,DUNE,ONION,0,-1,-1,0,2,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,120,-1,1175.0,720.000000,0.724628,2,BELT,-1,917,0,97,99,2,0.805062,5.434073
35996,99,2,328,-1,1226.0,640.000000,0.629288,2,CLOUD,-1,869,0,113,99,2,-0.189880,3.915998
35997,99,2,456,-1,1352.0,710.000000,0.601989,2,DIAGRAM,-1,1582,0,110,99,2,0.704887,5.686889
35998,99,2,655,-1,1175.0,500.000000,0.767687,2,GRASS,-1,1520,0,114,99,2,0.710150,8.478909


In [43]:
df_simu_g2.query("session == 0 and list == 0 and test == 1")

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,192,1402,1.0,213.237717,0.539325,1,BULB,SUIT,1,1,13,0,2,-1.0,-1.0
1,0,0,810,1481,0.0,1147.070679,0.455197,1,LEMON,TOOTH,0,-1,-1,0,2,-1.0,-1.0
2,0,0,384,686,1.0,10.579496,0.689500,1,COTTAGE,HANDKERCHIEF,1,1,8,0,2,-1.0,-1.0
3,0,0,1317,1137,0.0,19.992231,0.163475,1,SKILLET,PUBLISHER,0,-1,-1,0,2,-1.0,-1.0
4,0,0,506,977,0.0,2702.760742,0.408810,1,DUNE,ONION,0,-1,-1,0,2,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,1635,1509,1.0,158.322784,0.554214,1,ZEBRA,TROPHY,1,1,2,0,2,-1.0,-1.0
76,0,0,1490,1612,1.0,29.445311,0.638319,1,TOWER,WINDSHIELD,1,1,34,0,2,-1.0,-1.0
77,0,0,1153,1314,1.0,10.760128,0.688653,1,RACCOON,SKELETON,1,1,35,0,2,-1.0,-1.0
78,0,0,172,579,0.0,1941.372925,0.428888,1,BRICK,FLESH,0,-1,-1,0,2,-1.0,-1.0


In [44]:
subjects = np.unique(df_simu_g2.subject)
g2_stats = []
for subj in subjects:
    df_subj = df_simu_g2.query(f"subject == {subj}").copy()
    g2_stats.append(list(anal_perform(df_subj)))

In [45]:
print(np.array(g2_stats))
print(np.mean(np.array(g2_stats),axis=0))
print(sp.stats.sem(np.array(g2_stats),axis=0))

[[0.26666667 0.86666667 0.00833333 0.87336245]
 [0.53333333 0.83333333 0.00833333 0.84630739]
 [0.31666667 0.79166667 0.04166667 0.70442126]
 [0.25833333 0.76666667 0.         0.52598425]
 [0.24166667 0.85       0.00833333 0.21075903]
 [0.31666667 0.80833333 0.00833333 0.67234949]
 [0.23333333 0.825      0.01666667 0.88974557]
 [0.35       0.775      0.         0.77068153]
 [0.325      0.86666667 0.00833333 0.90430972]
 [0.3        0.81666667 0.         0.62867012]
 [0.25833333 0.775      0.025      0.78915329]
 [0.26666667 0.85       0.         0.67425968]
 [0.31666667 0.85       0.00833333 0.73956262]
 [0.24166667 0.76666667 0.         0.63305322]
 [0.29166667 0.825      0.         0.75605214]
 [0.30833333 0.775      0.00833333 0.83626009]
 [0.29166667 0.78333333 0.         0.812983  ]
 [0.29166667 0.825      0.01666667 0.5942648 ]
 [0.25833333 0.74166667 0.01666667 0.70386686]
 [0.45       0.80833333 0.01666667 0.89492326]
 [0.38333333 0.79166667 0.         0.78075777]
 [0.3        

In [46]:
# hack hr and far
test_num = 80
g2_stats_hack = np.array(g2_stats)
hacked_hr = (g2_stats_hack[:, 1] * test_num + 0.5) / (test_num + 0.5)
hacked_far = (g2_stats_hack[:, 2] * test_num + 0.5) / (test_num + 0.5)
g2_stats_hack[:, 1] = hacked_hr
g2_stats_hack[:, 2] = hacked_far
g2_stats_hack

array([[0.26666667, 0.86749482, 0.01449275, 0.87336245],
       [0.53333333, 0.83436853, 0.01449275, 0.84630739],
       [0.31666667, 0.79296066, 0.04761905, 0.70442126],
       [0.25833333, 0.76811594, 0.00621118, 0.52598425],
       [0.24166667, 0.85093168, 0.01449275, 0.21075903],
       [0.31666667, 0.80952381, 0.01449275, 0.67234949],
       [0.23333333, 0.82608696, 0.02277433, 0.88974557],
       [0.35      , 0.77639752, 0.00621118, 0.77068153],
       [0.325     , 0.86749482, 0.01449275, 0.90430972],
       [0.3       , 0.81780538, 0.00621118, 0.62867012],
       [0.25833333, 0.77639752, 0.0310559 , 0.78915329],
       [0.26666667, 0.85093168, 0.00621118, 0.67425968],
       [0.31666667, 0.85093168, 0.01449275, 0.73956262],
       [0.24166667, 0.76811594, 0.00621118, 0.63305322],
       [0.29166667, 0.82608696, 0.00621118, 0.75605214],
       [0.30833333, 0.77639752, 0.01449275, 0.83626009],
       [0.29166667, 0.78467909, 0.00621118, 0.812983  ],
       [0.29166667, 0.82608696,

In [47]:
# g2_stats = np.array(g2_stats)
g2_stats = g2_stats_hack
g2_ds = sp.stats.norm.ppf(np.array(g2_stats[:,1])) - sp.stats.norm.ppf(np.array(g2_stats[:,2]))
print(np.mean(g2_ds))
print(sp.stats.sem(g2_ds))

3.083272588091223
0.027821153546042993


### Item - CR (Group3)

In [16]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params)
df_simu_g3

CMR2 Time: 149.17164778709412


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,609,-1,1.0,402.298126,0.507586,1,FROST,NaN,1,0,14,0,3,-1.000000,-1.000000
1,0,0,101,-1,0.0,310.037628,0.300543,1,BATTERY,NaN,0,-1,-1,0,3,-1.000000,-1.000000
2,0,0,927,-1,0.0,2369.643555,0.402233,1,MOVIE,NaN,1,1,13,0,3,-1.000000,-1.000000
3,0,0,1303,-1,1.0,755.353516,0.476086,1,SIDEWALK,NaN,1,0,31,0,3,-1.000000,-1.000000
4,0,0,945,-1,1.0,763.326050,0.475561,1,NEST,NaN,1,0,2,0,3,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,99,3,509,-1,730.0,330.000000,0.758025,2,DUSTPAN,NaN,128,1,149,99,3,-0.192467,6.927903
47996,99,3,2,-1,1442.0,340.000000,0.739954,2,ACID,NaN,970,1,163,99,3,0.289813,6.711489
47997,99,3,1463,-1,327.0,520.000000,0.687198,2,TICK,NaN,1255,0,171,99,3,0.296310,5.322824
47998,99,3,1615,-1,1510.0,460.000000,0.648959,2,WINNER,NaN,667,1,157,99,3,0.203614,5.553589


In [17]:
subjects = np.unique(df_simu_g3.subject)
g3_stats = []
for subj in subjects:
    df_subj = df_simu_g3.query(f"subject == {subj}").copy()
    g3_stats.append(list(anal_perform(df_subj)))

In [18]:
print(np.array(g3_stats))
print(np.mean(np.array(g3_stats),axis=0))
print(sp.stats.sem(np.array(g3_stats),axis=0))

[[0.1        0.56875    0.3        0.80492988]
 [0.09375    0.6625     0.20625    0.69655549]
 [0.08125    0.61875    0.1875     0.3245283 ]
 [0.08125    0.66875    0.25625    0.64165103]
 [0.0875     0.59375    0.23125    0.41695416]
 [0.1125     0.625      0.25       0.49727005]
 [0.09375    0.55625    0.24375    0.51884701]
 [0.08125    0.575      0.25625    0.57924626]
 [0.0875     0.56875    0.18125    0.46153846]
 [0.10625    0.5875     0.2375     0.66833542]
 [0.0875     0.675      0.35       0.4469914 ]
 [0.08125    0.6125     0.2625     0.5215965 ]
 [0.0875     0.6        0.2125     0.40536513]
 [0.08125    0.58125    0.25625    0.7435763 ]
 [0.09375    0.59375    0.11875    0.45551933]
 [0.0875     0.68125    0.2625     0.43478261]
 [0.0875     0.65625    0.275      0.68267581]
 [0.075      0.61875    0.3375     0.4725987 ]
 [0.10625    0.625      0.21875    0.77131258]
 [0.075      0.6        0.28125    0.50429799]
 [0.09375    0.625      0.23125    0.73853904]
 [0.0875     

In [19]:
g3_stats = np.array(g3_stats)
g3_ds = sp.stats.norm.ppf(np.array(g3_stats[:,1])) - sp.stats.norm.ppf(np.array(g3_stats[:,2]))
print(np.mean(g3_ds))
print(sp.stats.sem(g3_ds))

0.9710540086959181
0.015336857574968176


In [20]:
stats = []
stats.append(list(np.mean(np.array(g1_stats),axis=0)))
stats.append(list(np.mean(np.array(g2_stats),axis=0)))
stats.append(list(np.mean(np.array(g3_stats),axis=0)))
stats = np.array(stats)
stats

array([[0.4041    , 0.8084    , 0.1546    , 0.79093977],
       [0.30675   , 0.8075    , 0.01      , 0.72957623],
       [0.0899375 , 0.611375  , 0.2485    , 0.57344004]])

In [21]:
ground_truth = np.array([[0.42, 0.72, 0.22, 0.81], [0.30, 0.80, 0.12, 0.71], [0.19, 0.67, 0.15, 0.57]])  # p_rc, hr, far, q
err = np.mean(np.power(stats - ground_truth,2))
err

0.0040380285103444056

### Performance Analysis

In [22]:
def array2df(group_stats, group_num):
    group_stats = np.array(group_stats)
    df = pd.DataFrame(group_stats, columns = ['PR','HR','FAR','Q'])
    df['d'] = sp.stats.norm.ppf(np.array(group_stats[:,1])) - sp.stats.norm.ppf(np.array(group_stats[:,2]))
    df['subject'] = df.index
    df['group'] = group_num
    return df
df_group1 = array2df(g1_stats, 1)
df_group2 = array2df(g2_stats, 2)
df_group3 = array2df(g3_stats, 3)
df_groups = pd.concat([df_group1, df_group2, df_group3])
df_groups

,PR,HR,FAR,Q,d,subject,group
0,0.57000,0.84000,0.14000,0.832100,2.074777,0,1
1,0.31000,0.76000,0.19000,0.822632,1.584199,1,1
2,0.32000,0.80000,0.23000,0.784314,1.580468,2,1
3,0.47000,0.77000,0.19000,0.846626,1.616743,3,1
4,0.34000,0.76000,0.12000,0.846921,1.881289,4,1
...,...,...,...,...,...,...,...
95,0.08750,0.57500,0.13125,0.768559,1.309620,95,3
96,0.10625,0.64375,0.24375,0.590846,1.062791,96,3
97,0.08750,0.62500,0.26250,0.357106,0.954296,97,3
98,0.10625,0.53125,0.20000,0.615444,0.920034,98,3


In [23]:
pg.anova(data=df_groups,dv='PR',between='group', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,5.172759,2,2.586379,567.560157,3.478373e-102,0.792615
1,Within,1.353433,297,0.004557,NaN,NaN,NaN


In [24]:
pg.pairwise_tukey(data=df_groups,dv='PR',between='group')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.40410,0.306750,0.097350,0.009547,10.197197,1.143530e-14,1.176607
1,1,3,0.40410,0.089938,0.314163,0.009547,32.907826,1.143530e-14,4.485620
2,2,3,0.30675,0.089938,0.216813,0.009547,22.710629,1.143530e-14,4.818664


In [25]:
pg.anova(data=df_groups.loc[np.isfinite(df_groups.d)], dv='d',between='group', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,185.215402,2,92.607701,2127.083899,6.093003e-164,0.941361
1,Within,11.537411,265,0.043537,NaN,NaN,NaN


In [26]:
pg.anova(data=df_groups,dv='Q',between='group', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,2.515004,2,1.257502,58.860479,2.934738e-22,0.283856
1,Within,6.345143,297,0.021364,NaN,NaN,NaN


In [27]:
pg.pairwise_tukey(data=df_groups,dv='Q',between='group')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.790940,0.729576,0.061364,0.020671,2.968609,9.050661e-03,0.435753
1,1,3,0.790940,0.573440,0.217500,0.020671,10.522072,1.143530e-14,1.509024
2,2,3,0.729576,0.573440,0.156136,0.020671,7.553463,1.591727e-12,1.008354


### Symmetry Analysis

In [28]:
def test_sym(df_simu, testnum):
    df = df_simu.query(f"test == {testnum} and order >= 0").copy()
    df['correct'] = df['s_resp'] == df['correct_ans']
    df_order = df.groupby(["order","subject"]).correct.mean().to_frame(name="p_correct").reset_index()
    return df_order

In [29]:
df_simu_all = pd.concat([df_simu_g1, df_simu_g2, df_simu_g3])
df_order_recog = test_sym(df_simu_all, 1)
print(df_order_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_recog, dv='p_correct', subject='subject', within='order')

order
0    0.750042
1    0.750375
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.08544,99.0,two-sided,0.932084,0.111,-0.011749


In [30]:
df_order_g1_recog = test_sym(df_simu_g1, 1)
print(df_order_g1_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_recog, dv='p_correct', subject='subject', within='order')

order
0    0.8293
1    0.8245
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,1.003083,99.0,two-sided,0.318266,0.18,0.132079


In [31]:
df_order_g1_cr = test_sym(df_simu_g1, 2)
print(df_order_g1_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_cr, dv='p_correct', subject='subject', within='order')

order
0    0.401983
1    0.406371
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.43808,99.0,two-sided,0.662282,0.122,-0.039487


In [32]:
df_order_g2_recog = test_sym(df_simu_g2, 1)
print(df_order_g2_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_recog, dv='p_correct', subject='subject', within='order')

order
0    0.812167
1    0.802833
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,1.447064,99.0,two-sided,0.151038,0.304,0.194566


In [33]:
df_order_g2_cr = test_sym(df_simu_g2, 2)
print(df_order_g2_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_cr, dv='p_correct', subject='subject', within='order')

order
0    0.3150
1    0.2985
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,2.455705,99.0,two-sided,0.015804,1.915,0.231


In [34]:
df_order_g3_recog = test_sym(df_simu_g3, 1)
print(df_order_g3_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_recog, dv='p_correct', subject='subject', within='order')

order
0    0.604375
1    0.618375
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-2.005849,99.0,two-sided,0.047601,0.755,-0.253369


In [35]:
df_order_g3_cr = test_sym(df_simu_g3, 2)
print(df_order_g3_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_cr, dv='p_correct', subject='subject', within='order')

order
0    0.088500
1    0.091375
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.701013,99.0,two-sided,0.48494,0.141,-0.127569
